In [1]:
# Project: Purchases focused on the season and the customer
# Purpose: Help a jewelry store decide what to purchase each season, based on inventory, budget, and material costs.
# Location: Guatemala City
# Author: [Eduardo Mart.]
# Date: 2025-06-02

import pandas as pd
import sqlite3
from datetime import datetime

# --- Configuration ---
# Define key parameters for the purchase planning
BUDGET_BY_MONTH = {
    'January': 60000,
    'March': 80000,
    'May': 150000,
    'July': 300000  # Forecast
}

STOCK_LEFT_PERCENTAGE = {
    'February': 0.35,
    'April': 0.28,
    'June': 0.13  # Forecast
}

SILVER_PRICE_PER_GRAM = 29
GOLD_PRICE_PER_GRAM = 500

SILVER_INVENTORY_KG = {
    'January': 6,
    'March': 10,
    'May': 12,
    'July': 15  # Forecast
}

GOLD_INVENTORY_GRAMS = {
    'March': 250,
    'July': 400  # Forecast
}

EXCEL_FILE_NAME = "jewelry_purchase_plan.xlsx"
DATABASE_FILE_NAME = "jewelry_store.db"
TABLE_NAME = "purchase_plan"

# --- Data Processing ---
def create_purchase_dataframe():
    """
    Creates a pandas DataFrame detailing the monthly jewelry purchase plan
    based on budget, material costs, and inventory.
    """
    records = []
    for month, budget in BUDGET_BY_MONTH.items():
        silver_grams = SILVER_INVENTORY_KG.get(month, 0) * 1000  # Convert kg to grams
        silver_cost = silver_grams * SILVER_PRICE_PER_GRAM

        gold_grams = GOLD_INVENTORY_GRAMS.get(month, 0)
        gold_cost = gold_grams * GOLD_PRICE_PER_GRAM

        total_material_cost = silver_cost + gold_cost
        remaining_budget = budget - total_material_cost

        # Include stock left percentage if available for the month
        stock_percentage = STOCK_LEFT_PERCENTAGE.get(month, 0.0)

        records.append({
            'Month': month,
            'Budget (Q)': budget,
            'Silver (g)': silver_grams,
            'Gold (g)': gold_grams,
            'Silver Cost (Q)': silver_cost,
            'Gold Cost (Q)': gold_cost,
            'Total Material Cost (Q)': total_material_cost,
            'Remaining Budget (Q)': remaining_budget,
            'Stock Left (%)': stock_percentage
        })
    return pd.DataFrame(records)

def export_to_excel(df, filename):
    """
    Exports the DataFrame to an Excel file.
    """
    try:
        df.to_excel(filename, index=False)
        print(f"Data successfully exported to {filename}")
    except Exception as e:
        print(f"Error exporting to Excel: {e}")

def create_and_populate_database(df, db_name, table_name):
    """
    Creates an SQLite database and populates a table with the DataFrame content.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Dynamically create CREATE TABLE SQL based on DataFrame columns
        # This makes the table creation more robust to changes in DataFrame structure
        columns = ", ".join([f"{col.replace(' ', '_').replace('(', '').replace(')', '')} REAL" if 'Cost' in col or 'Budget' in col or 'g' in col or '%' in col else f"{col.replace(' ', '_')} TEXT" for col in df.columns])
        create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns})"
        cursor.execute(create_table_sql)

        # Use to_sql for efficient DataFrame insertion
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        conn.commit()
        print(f"Data successfully saved to SQLite database '{db_name}' in table '{table_name}'")
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    finally:
        if conn:
            conn.close()

# --- Main Execution ---
if __name__ == "__main__":
    print("Generating Monthly Jewelry Purchase Plan...")

    purchase_df = create_purchase_dataframe()

    print("\nMonthly Jewelry Purchase Plan:")
    print(purchase_df)

    export_to_excel(purchase_df, EXCEL_FILE_NAME)
    create_and_populate_database(purchase_df, DATABASE_FILE_NAME, TABLE_NAME)

    print("\n--- Process Complete ---")
    print("Next steps: Visualize the data in Tableau or Power BI for deeper insights.")

Generating Monthly Jewelry Purchase Plan...

Monthly Jewelry Purchase Plan:
     Month  Budget (Q)  Silver (g)  Gold (g)  Silver Cost (Q)  Gold Cost (Q)  \
0  January       60000        6000         0           174000              0   
1    March       80000       10000       250           290000         125000   
2      May      150000       12000         0           348000              0   
3     July      300000       15000       400           435000         200000   

   Total Material Cost (Q)  Remaining Budget (Q)  Stock Left (%)  
0                   174000               -114000             0.0  
1                   415000               -335000             0.0  
2                   348000               -198000             0.0  
3                   635000               -335000             0.0  
Error exporting to Excel: No module named 'openpyxl'
SQLite error: near "%": syntax error

--- Process Complete ---
Next steps: Visualize the data in Tableau or Power BI for deeper insight